# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"s
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

3


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead' # ''ArcHead', CosHead', 'SphereHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception
#     But if you are trying to use NasNet, please check this issue first: https://github.com/keras-team/keras-applications/issues/78
#         We manually download the weight file and explicitly load it in models.py file
# Backbones w/o pretrained weights:
#     MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
backbone_type = 'EfficientNetLite0' 
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653
if head_type == 'SphereHead':
    base_lr = 0.01 
#     margin = 1.35
#     logist_scale = 30.0 
    margin = 4
    logist_scale = 1
elif head_type == 'CosHead':
    base_lr = 0.01 
    margin=0.35
    logist_scale=64
elif head_type == 'ArcHead':
    base_lr = 0.01 
    margin=0.5
    logist_scale=64
else:
    base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print(train_size)
steps_per_epoch = train_size // batch_size
print(steps_per_epoch)
val_size = dataset_len - train_size
print(val_size)
validation_steps = val_size // batch_size
print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False   # True

dgx = True

4658122
36391
1164531
9097


In [4]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

# print("data: ", train_dataset)

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             margin=margin, 
                             logist_scale=logist_scale,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.summary()

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413024     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
#                               patience=0.1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 1:37:38 - loss: 46.4101
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_1_l_46.313289642333984.ckpt
 2000/36391 [>.............................] - ETA: 1:31:08 - loss: 46.2927
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_1_l_46.02696990966797.ckpt
 3000/36391 [=>............................] - ETA: 1:27:03 - loss: 46.1537
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_1_l_45.72218704223633.ckpt
 4000/36391 [==>...........................] - ETA: 1:26:00 - loss: 46.0067
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_1_l_45.41082763671875.ckpt
 5000/36391 [===>...........

### Resume training with latest checkpoint

In [ ]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

In [ ]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

In [ ]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)